# Configure Python Path
Add the `src/` folder to the Python path so that modules like `llm_interaction`, `validator`, or `utils` can be imported from the source directory.

In [2]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# Define or Load a Scenario
Enter the scenario description that will be used aas input for the LLM. You can write it manually or load it from a `.txt` file.

In [3]:
# Option 1: Directly define the scenario (uncomment the next line and modify it)
# scenario = "Write your scenario here"

# Option 2: Load the scenario from a file
from pathlib import Path

scenario_path = Path("..") / "tasks" / "1.txt"
with open(scenario_path, "r", encoding="utf-8") as file:
    scenario = file.read()

# Generate Prompt Context
Use the scenario and selected protocols to retrieve relevant context from the database. This helps the LLM generate more accurate and aligned outputs.

Set the number of contextual examples to include in the prompt, and define the list of relevant protocols involved in the scenario.

In [4]:
number_examples = 1
protocol_list = ["ocpp"]

In [5]:
from generate_prompt import generate_prompt

prompt = generate_prompt(scenario=scenario, protocol_list=protocol_list, num_examples=number_examples)

print(prompt)


You are an AI that generates XML properties for deep packet inspection.

### MMT Property Context:
XML Rules:
- A <property> defines a security rule (SECURITY_RULE) or an attack rule (ATTACK).
- Each property begins with a <property> tag and ends with </property>
- A property is a "general ordered tree"
- The nodes of the property tree are:
  - <property> node (required)
  - <operator> nodes (optional)
  - <event> nodes (required)
- The <property> node is forcibly the root node and the <event> nodes are forcibly leaf nodes.
- The left branch represents the context and the right branch represents the trigger.
- The property is found valid when the trigger is found valid; and the trigger is checked only if the context is valid.

- <beginning> tag:
  - Container tag for defining multiple properties
  - Holds one or more <property> definitions
  - Includes optional embedded functions.

- A <property> tag contains:
  - value (optional): THEN, defines the logical relationship between contex

# Configure the LLM
Set the model name and parameters that will be used to initialize the LLM, such as temperature, top-p, and max tokens.

Specify the LLM model (e.g., `gemma3:12b`) and its generation parameters such as temperature and top-p.

In [6]:
model_name = "gemma3:12b"
model_config = {"provider": "ollama", "temperature": 0.7, "top_p": 0.9, "max_tokens": 1024}

# Initialize the LLM
Load and prepare the LLM model using the configuration specified above.

In [7]:
from llm_interaction import initialize_llm

llm = initialize_llm(model_name=model_name, model_config=model_config)

Define how many refinement attempts the LLM is allowed in case the output needs to be corrected based on validation feedback

In [8]:
max_iterations = 5

# Run the scenario
Execute the full LLM interaction loop: generate a response, extract XML, validate it, and optionally refine it based on feedback.

In [9]:
from llm_interaction import generate_property
generated_property, is_valid = generate_property(llm, prompt, max_iterations=max_iterations)
if is_valid:
    print("Generated property:")
    print(generated_property)
else:
    print("Failed to generate a valid property.")

Iteration 1 of 5
Invalid XML generated: Invalid type_property attribute in property tag
Iteration 2 of 5
Invalid XML generated: Invalid XML syntax
Iteration 3 of 5
Valid XML generated:
<beginning>
  <property value="THEN" delay_units="ms" delay_min="0" delay_max="0" property_id="101" type_property="ATTACK"
             description="OCPP Charging Authorisation Request Timeout">
    <event value="COMPUTE" event_id="1"
            description="Charging Authorisation Request"
            boolean_expression="(ocpp.messageType == 'ChargingAuthorisationRequest')"/>
    <event value="COMPUTE" event_id="2"
            description="Timeout Detected"
            boolean_expression="(#em_ocpp_timeout_check(ocpp.timestamp))"/>
  </property>

  <embedded_functions><![CDATA[
#include <stdlib.h>
#include <mmt_core.h>
#include <tcpip/mmt_tcpip.h>
#include <time.h>

static unsigned long long timeout_threshold_ms = 5000; // Default timeout: 5 seconds

void on_load() {
  const char *str = getenv("OCPP_Cha